In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__init__"), '..')))
from src.utils import * 
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
basePath = "ECG_Database"

## List files and folders insade basePath
patDir = sorted(
    [f.path for f in os.scandir(basePath) if f.is_dir()],
    key=lambda x: int(os.path.basename(x).replace("patient", ""))  # Extraer el número de paciente
)

## Dictionary to store each patient's records
patDict = {}

#def generateName(i, extension):
        #return f"record{i:03d}{extension}" 

for patient in patDir:
    patID=os.path.basename(patient) # patID = directory name
    archivos = os.listdir(patient)

    # Create 3 lists, one for each file type 
    dat = [f for f in archivos if f.endswith(".dat")]
    hea = [f for f in archivos if f.endswith(".hea")]
    xyz = [f for f in archivos if f.endswith(".xyz")]

    # Rename the files for clarity
    
    import os

    # Agrupar los archivos por nombre base y asignarles un número secuencial
    allFiles = []
    
    # Agrupar los archivos con el mismo nombre base
    for f in dat + hea + xyz: 
        allFiles.append(f)  # Asignar el nuevo nombre

    patDict[patID] = {"files": allFiles}

In [ ]:
for patient, data in patDict.items():
    print(f"Paciente: {patient}")
    
    # Imprimir los archivos del paciente
    if "files" in data:
        print("Archivos:", ", ".join(data["files"]))
    else:
        print("No hay archivos registrados.")
    
    print("-" * 40)

In [ ]:
allFeatures = {}

for patient_id, data in patDict.items():
    archivos = data["files"]
    dat_files = [f for f in archivos if f.endswith(".dat")]
    
    patient_features = {}  # Dict para todos los archivos de este paciente
    
    for archivo in dat_files:
        features = extract_features(patient_id, archivo)
        patient_features.update(features[patient_id])  # Extraemos solo el contenido interno
    
    allFeatures[patient_id] = patient_features

In [ ]:
infarct=possible_infarct("patient001", "s0010_re.dat", allFeatures)
arrhythmia=possible_arrhythmia("patient001", "s0010_re.dat", allFeatures)
branch_block=possible_branch_block("patient001", "s0010_re.dat", allFeatures)
ventricular_hypertrophy=possible_ventricular_hypertrophy("patient001", "s0010_re.dat", allFeatures)
long_QT_syndrome=possible_long_QT_syndrome("patient001", "s0010_re.dat", allFeatures)
brugada_syndrome=possible_brugada_syndrome("patient001", "s0010_re.dat", allFeatures)
pericarditis=possible_pericarditis("patient001", "s0010_re.dat", allFeatures)

print(f"possible_infarct:{infarct}, possible_arrhythmia:{arrhythmia}, possible_branch_block:{branch_block}, possible_ventricular_hypertrophy:{ventricular_hypertrophy}, possible_long_QT_syndrome:{long_QT_syndrome}, possible_brugada_syndrome:{brugada_syndrome}, possible_pericarditis:{pericarditis}")